In [1]:
import glob
import os
import pathlib

import numpy as np
import pandas as pd

from util.image import unnormalize

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import functional as F

from torchvision import transforms
from torchinfo import torchinfo
from tqdm import tqdm
import matplotlib.pyplot as plt

import albumentations as A
import torch.nn.functional as F

from PIL import Image

import torchmetrics
from torchvision.utils import save_image, make_grid
import cv2
from util.io import load_ckpt

from util.loss import  InpaintingLoss
import os, glob


device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

In [2]:
facemask_dataset = 'facemask bounding box/images/'
facemask_annotations = 'facemask bounding box/annotations/'

In [3]:
glob.glob(facemask_dataset)

[]

In [4]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
	("mouth", (48, 68)),
	("right_eyebrow", (17, 22)),
	("left_eyebrow", (22, 27)),
	("right_eye", (36, 42)),
	("left_eye", (42, 48)),
	("nose", (27, 35)),
	("jaw", (0, 17))
])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 FACIAL_LANDMARKS_IDXS = OrderedDict([                                                       │
│    2 │   ("mouth", (48, 68)),                                                                    │
│    3 │   ("right_eyebrow", (17, 22)),                                                            │
│    4 │   ("left_eyebrow", (22, 27)),                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'OrderedDict' is not defined

In [5]:
cv2.data.haarcascades

'c:\\ProgramData\\Anaconda3\\envs\\ml\\lib\\site-packages\\cv2\\data\\'

In [6]:
import cv2

# Load the cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
nose_cascade = cv2.CascadeClassifier( cv2.data.haarcascades + 'haarcascade_mcs_nose.xml')

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

while True:
    # Read the frame
    _, img = cap.read()
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

    for (x, y, w, h) in eyes:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1)
    # Display
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
# Release the VideoCapture object
cap.release()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 │   # Convert to grayscale                                                                  │
│   17 │   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                                            │
│   18 │   # Detect the faces                                                                      │
│ ❱ 19 │   faces = face_cascade.detectMultiScale(gray, 1.1, 4)                                     │
│   20 │   eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)                                       │
│   21 │   # Draw the rectangle around each face                                                   │
│   22 │   for (x, y, w, h) in faces:                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt